In [ ]:
import pandas as pd
import numpy as np
! pip install papermill
! pip install recommenders[examples,gpu]
from recommenders.datasets.split_utils import min_rating_filter_pandas
!pip3 install scikit-surprise
import surprise
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from surprise import accuracy
from recommenders.utils.python_utils import binarize
from collections import defaultdict
import random
from surprise import Dataset
from surprise.model_selection import GridSearchCV
from prettytable import PrettyTable

In [ ]:
my_seed = 0
random.seed(my_seed)
np.random.seed(my_seed)

In [ ]:
df = pd.DataFrame()
df[["userID", "itemID", "rating", "timestamp"]] = pd.read_json("AMAZON_FASHION_5.json")[["reviewerID", "asin", "overall", "unixReviewTime"]]
df.head()

,userID,itemID,rating,timestamp
0,ALJ66O1Y6SLHA,B000K2PJ4K,5,144132480
1,ALJ66O1Y6SLHA,B000K2PJ4K,5,144132480
2,ALJ66O1Y6SLHA,B000K2PJ4K,5,144132480
3,ALJ66O1Y6SLHA,B000K2PJ4K,5,144132480
4,ALJ66O1Y6SLHA,B000K2PJ4K,5,144132480


In [ ]:
df.shape

(3176, 4)

In [ ]:
df['rating'].value_counts()

5    2158
4     471
3     337
1     117
2      93
Name: rating, dtype: int64

In [ ]:
sparsity = lambda dataf: dataf.isna().sum().sum() / (dataf.shape[0] * dataf.shape[1])

In [ ]:
usercount = df[['userID']].groupby('userID', as_index = False).size()
itemcount = df[['itemID']].groupby('itemID', as_index = False).size()

density = 1. * df.shape[0] / (usercount.shape[0] * itemcount.shape[0])

print("After filtering, there are %d ratings from %d users on %d products (sparsity: %.3f%%)" % 
      (df.shape[0], usercount.shape[0], itemcount.shape[0], (1 - density) * 100))


After filtering, there are 3176 ratings from 406 users on 31 products (sparsity: 74.766%)


In [ ]:
df.rating.value_counts()

5    2158
4     471
3     337
1     117
2      93
Name: rating, dtype: int64

In [ ]:
def GetTopN(predictions, n, minimumRating, criterion):
    topN = defaultdict(list)
    
    for index, row in predictions.iterrows():
        if (row[criterion] >= minimumRating):
            topN[(row.uid)].append(((row.iid), row[criterion]))

    for userID, ratings in topN.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        topN[(userID)] = ratings[:n]

    return topN

In [ ]:
def DCG(query_relevancy_labels, k):
    # Use log with base 2
    value=min(k,len(query_relevancy_labels))
    sum1=0
    for i in range(value):
      sum1=sum1+((query_relevancy_labels[i])/(np.log2(2+i)))

    return sum1

def NDCG(query_relevancy_labels, k):
    sorted_list= np.sort(query_relevancy_labels)[::-1]
    dcg1 = DCG(query_relevancy_labels, k)
    dcg2 = DCG(sorted_list, k)
    if dcg2 == 0:
      return 0
    else:
      return dcg1/dcg2

In [ ]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [ ]:
def ndcg_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    dcg = dict()
    ndcg = dict()
    for uid, user_ratings in user_est_true.items():
        rel=[]

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        for _, true_r in user_ratings:
          if true_r>=threshold:
            rel.append(true_r)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.
        #print(rel)

        ndcg[uid] = NDCG(rel, k)

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        #recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return ndcg

In [ ]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

In [ ]:
raw_ratings = data.raw_ratings
random.shuffle(raw_ratings)
threshold = int(.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]
data.raw_ratings = A_raw_ratings

In [ ]:
sim_options = {'name': 'cosine',
               'user_based': False  
               }
param_grid = {'k': [10, 20, 30, 40, 50, 60, 70, 80, 90], 'min_k': [10, 20, 30, 40, 50, 60, 70, 80, 90]
              }
gs = GridSearchCV(KNNBaseline, param_grid, cv=3)

gs.fit(data)


Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

In [ ]:
gs.best_params['rmse']['k']

10

In [ ]:
sim_options = {'name': 'pearson_baseline',
               'user_based': False  
               }
algo=KNNBaseline(k=gs.best_params['rmse']['k'], min_k=gs.best_params['rmse']['min_k'],sim_options=sim_options)
trainset = data.build_full_trainset()
algo.fit(trainset)

# Compute biased accuracy on A
predictions = algo.test(trainset.build_testset())
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions)

# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)



Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Biased accuracy on A,   RMSE: 0.8066
Unbiased accuracy on B, RMSE: 0.8898


0.8898435444271433

In [ ]:
newTable = PrettyTable(["Algorithm", "Sparsity", "Precision@k", "Recall@k", "NDCG@k"]) 
precisions, recalls = precision_recall_at_k(predictions, k=100, threshold=4)
ndcg_val = ndcg_at_k(predictions, k=100, threshold=4)
precisionk=(sum(prec for prec in precisions.values()) / len(precisions))
recallk=(sum(rec for rec in recalls.values()) / len(recalls))
ndcgk=(sum(nd for nd in ndcg_val.values()) / len(ndcg_val))
sparse=0.70
newTable.add_row(["KNN_Baseline", sparse, precisionk , recallk, ndcgk])   
print(newTable)

+--------------+----------+--------------------+--------------------+--------------------+
|  Algorithm   | Sparsity |    Precision@k     |      Recall@k      |       NDCG@k       |
+--------------+----------+--------------------+--------------------+--------------------+
| KNN_Baseline |   0.7    | 0.7894736842105263 | 0.7894736842105263 | 0.8011695906432749 |
+--------------+----------+--------------------+--------------------+--------------------+


In [ ]:
def _print(message, verbose):
    if verbose:
        print(message)
    

def load_dataset(fp):
    df = pd.DataFrame()
    try:
        df[["userID", "itemID", "rating", "timestamp"]] = pd.read_json(fp)[["reviewerID", "asin", "overall", "unixReviewTime"]]
    except:
        df = pd.read_json(fp)
    return df


def filter_to_sparsity(df, sparsity_percentage, verbose = False):
    # Obtain both usercount and itemcount after filtering
    usercount = df[['userID']].groupby('userID', as_index = False).size()
    itemcount = df[['itemID']].groupby('itemID', as_index = False).size()

    sparsity = 1 - (df.shape[0] / (usercount.shape[0] * itemcount.shape[0]))

    _print(f"After filtering, there are {df.shape[0]} ratings from {usercount.shape[0]} users on {itemcount.shape[0]}" + 
           f" products (sparsity: {sparsity * 100:.3f})", verbose)
    
    drop_item_ratings = int(-((1-sparsity_percentage) * (usercount.shape[0] * itemcount.shape[0]) - df.shape[0]))
    print(f"To obtain a sparsity of {sparsity_percentage * 100}% we need to drop {drop_item_ratings} ratings")
    drop_indices = np.random.choice(df.index, size=drop_item_ratings)
    df.drop(drop_indices, inplace=True)

    sparsity = 1 - (df.shape[0] / (usercount.shape[0] * itemcount.shape[0]))
    _print(f"After dropping cells, there are {df.shape[0]} ratings from {usercount.shape[0]} users on {itemcount.shape[0]}" + 
           f" products (sparsity: {sparsity * 100:.3f})", verbose)
    return df


In [ ]:
def build_data(df):
    
    reader = Reader(rating_scale=(1,5))
    data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)
    return data

In [ ]:
def data_loading_pipeline(fp, sparsity_percentage=None, with_writing=False):
    """
    fp: str = Filepointer to desired user-item-ratings json.
    sparsity_percentage: float = value between 0-1.
    """
    df = load_dataset(fp)
    #df = filter_on_minimal_ratings(df)
    df = filter_to_sparsity(df, sparsity_percentage)
    data = build_data(df)
    
    return data




In [ ]:
data = data_loading_pipeline("AMAZON_FASHION_5.json", 0.75)



raw_ratings = data.raw_ratings
random.shuffle(raw_ratings)
threshold = int(.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]
data.raw_ratings = A_raw_ratings

sim_options = {'name': 'pearson_baseline',
               'user_based': False  
               }

param_grid = {'k': [10, 20, 30, 40, 50, 60, 70, 80, 90], 'min_k': [10, 20, 30, 40, 50, 60, 70, 80, 90]
              }
gs = GridSearchCV(KNNBaseline, param_grid, cv=3)

gs.fit(data)

algo=KNNBaseline(sim_options=sim_options,k=gs.best_params['rmse']['k'], min_k=gs.best_params['rmse']['min_k'] )

trainset = data.build_full_trainset()
algo.fit(trainset)

# Compute biased accuracy on A
predictions = algo.test(trainset.build_testset())
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions)

# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)



newTable = PrettyTable(["Algorithm", "Sparsity", "Precision@k", "Recall@k", "NDCG@k"]) 
precisions, recalls = precision_recall_at_k(predictions, k=100, threshold=4)
ndcg_val = ndcg_at_k(predictions, k=100, threshold=4)
precisionk=(sum(prec for prec in precisions.values()) / len(precisions))
recallk=(sum(rec for rec in recalls.values()) / len(recalls))
ndcgk=(sum(nd for nd in ndcg_val.values()) / len(ndcg_val))
sparse=0.75
newTable.add_row(["KNN_Baseline", sparse, precisionk , recallk, ndcgk])   
print(newTable)


To obtain a sparsity of 75.0% we need to drop 29 ratings
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimat

In [ ]:
data = data_loading_pipeline("AMAZON_FASHION_5.json", 0.8)



raw_ratings = data.raw_ratings
random.shuffle(raw_ratings)
threshold = int(.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]
data.raw_ratings = A_raw_ratings

sim_options = {'name': 'pearson_baseline',
               'user_based': False  
               }
algo=KNNBaseline(sim_options=sim_options, k=gs.best_params['rmse']['k'], min_k=gs.best_params['rmse']['min_k'])

trainset = data.build_full_trainset()
algo.fit(trainset)

# Compute biased accuracy on A
predictions = algo.test(trainset.build_testset())
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions)

# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)



newTable = PrettyTable(["Algorithm", "Sparsity", "Precision@k", "Recall@k", "NDCG@k"]) 
precisions, recalls = precision_recall_at_k(predictions, k=100, threshold=4)
ndcg_val = ndcg_at_k(predictions, k=100, threshold=4)
precisionk=(sum(prec for prec in precisions.values()) / len(precisions))
recallk=(sum(rec for rec in recalls.values()) / len(recalls))
ndcgk=(sum(nd for nd in ndcg_val.values()) / len(ndcg_val))
newTable.add_row(["KNN_Baseline", 0.8, precisionk , recallk, ndcgk])   
print(newTable)


To obtain a sparsity of 80.0% we need to drop 658 ratings
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Biased accuracy on A,   RMSE: 0.7400
Unbiased accuracy on B, RMSE: 0.8041
+--------------+----------+--------------------+--------------------+--------------------+
|  Algorithm   | Sparsity |    Precision@k     |      Recall@k      |       NDCG@k       |
+--------------+----------+--------------------+--------------------+--------------------+
| KNN_Baseline |   0.8    | 0.8051282051282052 | 0.8051282051282052 | 0.8205128205128205 |
+--------------+----------+--------------------+--------------------+--------------------+


In [ ]:
data = data_loading_pipeline("AMAZON_FASHION_5.json", 0.85)



raw_ratings = data.raw_ratings
random.shuffle(raw_ratings)
threshold = int(.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]
data.raw_ratings = A_raw_ratings

sim_options = {'name': 'pearson_baseline',
               'user_based': False  
               }
algo=KNNBaseline(sim_options=sim_options, k=gs.best_params['rmse']['k'], min_k=gs.best_params['rmse']['min_k'])

trainset = data.build_full_trainset()
algo.fit(trainset)

# Compute biased accuracy on A
predictions = algo.test(trainset.build_testset())
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions)

# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)



newTable = PrettyTable(["Algorithm", "Sparsity" ,"Precision@k", "Recall@k", "NDCG@k"]) 
precisions, recalls = precision_recall_at_k(predictions, k=100, threshold=4)
ndcg_val = ndcg_at_k(predictions, k=100, threshold=4)
precisionk=(sum(prec for prec in precisions.values()) / len(precisions))
recallk=(sum(rec for rec in recalls.values()) / len(recalls))
ndcgk=(sum(nd for nd in ndcg_val.values()) / len(ndcg_val))
newTable.add_row(["KNN_Baseline", 0.85, precisionk , recallk, ndcgk])   
print(newTable)


To obtain a sparsity of 85.0% we need to drop 1288 ratings
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Biased accuracy on A,   RMSE: 0.7685
Unbiased accuracy on B, RMSE: 0.8767
+--------------+----------+--------------------+--------------------+--------------------+
|  Algorithm   | Sparsity |    Precision@k     |      Recall@k      |       NDCG@k       |
+--------------+----------+--------------------+--------------------+--------------------+
| KNN_Baseline |   0.85   | 0.7894736842105263 | 0.7894736842105263 | 0.8011695906432749 |
+--------------+----------+--------------------+--------------------+--------------------+


In [ ]:
data = data_loading_pipeline("AMAZON_FASHION_5.json", 0.9)



raw_ratings = data.raw_ratings
random.shuffle(raw_ratings)
threshold = int(.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]
data.raw_ratings = A_raw_ratings

sim_options = {'name': 'pearson_baseline',
               'user_based': True  
               }
algo=KNNBaseline(sim_options=sim_options, k=gs.best_params['rmse']['k'], min_k=gs.best_params['rmse']['min_k'])

trainset = data.build_full_trainset()
algo.fit(trainset)

# Compute biased accuracy on A
predictions = algo.test(trainset.build_testset())
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions)

# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)



newTable = PrettyTable(["Algorithm", "Sparsity", "Precision@k", "Recall@k", "NDCG@k"]) 
precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)
ndcg_val = ndcg_at_k(predictions, k=5, threshold=4)
precisionk=(sum(prec for prec in precisions.values()) / len(precisions))
recallk=(sum(rec for rec in recalls.values()) / len(recalls))
ndcgk=(sum(nd for nd in ndcg_val.values()) / len(ndcg_val))
newTable.add_row(["KNN_Baseline", 0.9, precisionk , recallk, ndcgk])   
print(newTable)

To obtain a sparsity of 90.0% we need to drop 1917 ratings
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Biased accuracy on A,   RMSE: 0.2378
Unbiased accuracy on B, RMSE: 0.4708
+--------------+----------+--------------------+--------------------+-------------------+
|  Algorithm   | Sparsity |    Precision@k     |      Recall@k      |       NDCG@k      |
+--------------+----------+--------------------+--------------------+-------------------+
| KNN_Baseline |   0.9    | 0.7602739726027398 | 0.7602739726027398 | 0.863013698630137 |
+--------------+----------+--------------------+--------------------+-------------------+


In [ ]:
data = data_loading_pipeline("AMAZON_FASHION_5.json", 0.95)



raw_ratings = data.raw_ratings
random.shuffle(raw_ratings)
threshold = int(.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]
data.raw_ratings = A_raw_ratings

sim_options = {'name': 'pearson_baseline',
               'user_based': True 
               }
algo=KNNBaseline(sim_options=sim_options, k=gs.best_params['rmse']['k'], min_k=gs.best_params['rmse']['min_k'])

trainset = data.build_full_trainset()
algo.fit(trainset)

# Compute biased accuracy on A
predictions = algo.test(trainset.build_testset())
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions)

# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)



newTable = PrettyTable(["Algorithm", "Sparsity", "Precision@k", "Recall@k", "NDCG@k"]) 
precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)
ndcg_val = ndcg_at_k(predictions, k=5, threshold=4)
precisionk=(sum(prec for prec in precisions.values()) / len(precisions))
recallk=(sum(rec for rec in recalls.values()) / len(recalls))
ndcgk=(sum(nd for nd in ndcg_val.values()) / len(ndcg_val))
newTable.add_row(["KNN_Baseline", 0.95, precisionk , recallk, ndcgk]) 
print(newTable)  


To obtain a sparsity of 95.0% we need to drop 2546 ratings
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Biased accuracy on A,   RMSE: 0.3244
Unbiased accuracy on B, RMSE: 0.6649
+--------------+----------+--------------------+--------------------+--------------------+
|  Algorithm   | Sparsity |    Precision@k     |      Recall@k      |       NDCG@k       |
+--------------+----------+--------------------+--------------------+--------------------+
| KNN_Baseline |   0.95   | 0.7142857142857143 | 0.7142857142857143 | 0.8151260504201681 |
+--------------+----------+--------------------+--------------------+--------------------+
